# NAIVE LEARNING

In [ ]:
import wandb

from sage.config import *
from sage.training import runner
from utils.misc import seed_everything

In [ ]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['IXI', 'Dallas', 'Oasis1']
cfg.plot = False

cfg.augment = False
cfg.epochs = 500
cfg.model_name = 'resnet_no_maxpool'
cfg.early_patience = 30
# cfg.start_channels = 8

In [ ]:
name = 'MNI / OAS3 ONLY / EXTRA_EPOCHS / Resnet_no_maxpool'

In [ ]:
wandb.login()
wandb.init(project='3d_smri',
           config=vars(cfg),
           name=name
  )

In [ ]:
model = runner.run(cfg)

In [ ]:
wandb.finish()

# UNLEARNING

In [1]:
import wandb

from sage.config import *
from sage.training import unlearner
from utils.misc import seed_everything

In [2]:
cfg = load_config()
seed_everything(cfg.seed)
cfg.registration = 'mni'
cfg.unused_src = ['Oasis1', 'Oasis3']

cfg.unlearn = True
cfg.unlearn_cfg.domainer.num_dbs = 4 - len(cfg.unused_src)

In [3]:
name = 'UNLEARNING TESTBED'

In [4]:
wandb.login()
wandb.init(project='3d_smri_unlearning',
           config=vars(cfg),
           name=name
  )

wandb: Currently logged in as: 1pha (use `wandb login --relogin` to force relogin)


In [ ]:
unlearner.run(cfg)

Use cuda:0 as a device.
Output from encoder is 128.
TOTAL TRAIN 448 | VALID 112
Epoch 1 / 100, BEST MAE inf
[train] 27.8 sec [valid] 2.7 sec 
TRAIN :: REG 3171.842 DOM 0.689 CONF 111.558 
RMSE 56.32 | MAE 53.34 | R2 -8.66 | CORR 0.30
TRAIN :: REG 2689.167 DOM 0.699 CONF 111.120 
RMSE 51.86 | MAE 49.14 | R2 -8.79 | CORR 0.18
Epoch 2 / 100, BEST MAE 49.139
[train] 26.5 sec [valid] 2.8 sec 
TRAIN :: REG 3146.647 DOM 0.669 CONF 112.826 
RMSE 56.09 | MAE 53.12 | R2 -8.59 | CORR 0.34
TRAIN :: REG 2670.912 DOM 0.683 CONF 111.032 
RMSE 51.68 | MAE 48.95 | R2 -8.73 | CORR -0.21
Epoch 3 / 100, BEST MAE 48.946
[train] 26.8 sec [valid] 2.8 sec 
TRAIN :: REG 3117.849 DOM 0.633 CONF 116.798 
RMSE 55.84 | MAE 52.86 | R2 -8.50 | CORR 0.34
TRAIN :: REG 2648.062 DOM 0.622 CONF 115.652 
RMSE 51.46 | MAE 48.76 | R2 -8.64 | CORR 0.42
Epoch 4 / 100, BEST MAE 48.756
[train] 26.8 sec [valid] 2.9 sec 
TRAIN :: REG 3078.371 DOM 0.598 CONF 124.150 
RMSE 55.48 | MAE 52.51 | R2 -8.38 | CORR 0.40
TRAIN :: REG 2618.